In [ ]:
!pip install -q transformers torch scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving dataset-merged.csv.zip to dataset-merged.csv (1).zip


,Unnamed: 0,text,label,wcount
0,0,‘मोदी के शासन के दौरान गंगा’ गंगा नदी नरेन्द्...,1,19
1,1,यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान/...,1,374
2,2,गुलाब गेंद वाल डे-नाइट टेस्ट मैच कप्ता विराट क...,0,20
3,3,उत्तर कोरिया रॉकेट प्रक्षेपण योजनाएं 71 0 15 0...,1,345
4,4,राष्ट्रपति डोनाल्ड ट्रम्प और प्रथम महिला मेलान...,0,180


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "Sailesh97/Hinvec"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True
)

model.eval()

BidirectionalMistralModel(
  (embed_tokens): Embedding(32768, 2048, padding_idx=0)
  (layers): ModuleList(
    (0-15): 16 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=2048, out_features=7168, bias=False)
        (up_proj): Linear(in_features=2048, out_features=7168, bias=False)
        (down_proj): Linear(in_features=7168, out_features=2048, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm((2048,), eps=1e-06)
      (post_attention_layernorm): MistralRMSNorm((2048,), eps=1e-06)
    )
  )
  (norm): MistralRMSNorm((2048,), eps=1e-06)
  (rotary_emb): MistralRotaryEmbedd

In [ ]:
def get_hinvec_embedding(text):
    tokens = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=32
    )

    # ⛔ DO NOT call model(**tokens)
    # ✅ Directly use embedding layer
    with torch.no_grad():
        token_embeddings = model.get_input_embeddings()(tokens["input_ids"])

    # Mean pooling
    embedding = token_embeddings.mean(dim=1)

    return embedding.squeeze().cpu().numpy()

In [ ]:
X = []
y = df["label"].values

for text in df["text"].astype(str):
    emb = get_hinvec_embedding(text)
    X.append(emb)

X = np.array(X)
print(X.shape)

(17124, 2048)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7646715328467153


In [ ]:
# Select one FAKE news from dataset
fake_news_row = df[df["label"] == 1].iloc[0]

news_text = fake_news_row["text"]

# Generate embedding
news_embedding = get_hinvec_embedding(news_text)

# Predict
prediction = clf.predict([news_embedding])[0]

print("News Text:")
print(news_text)

print("\nModel Prediction:")
if prediction == 1:
    print("🛑 Fake News")
else:
    print("✅ Real News")

News Text:
‘मोदी के शासन के दौरान गंगा’  गंगा नदी नरेन्द्र मोदी के शाशन में कांग्रेस के मुकाबले काफ़ी स्वच्छ है  

Model Prediction:
✅ Real News


In [ ]:
test_news =  "यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान/गेटी द रियल एजेंडा न्यूज जेम्स ओकीफ के प्रोजेक्ट वेरिटास की जांच के परिणामस्वरूप डीएनसी ओबामा व्हाइट हाउस और हिलेरी क्लिंटन अभियान के बीच भ्रष्टाचार के कभी न खत्म होने वाले स्तर के बारे में अधिक विवरण सामने आए हैं। ट्रम्प की रैलियों में लोगों को बत्तख के रूप में तैयार करने के लिए प्रतीत होने वाले हानिरहित अभियानों को बढ़ावा देने से लेकर क्लिंटन अभियान डीएनसी और गैर-लाभकारी संगठन अमेरिकन्स यूनाइटेड फॉर चेंज के बीच मिलीभगत तक भ्रष्ट आचरण शामिल हैं। प्रोजेक्ट वेरिटास द्वारा डाले गए तीसरे वीडियो में हिलेरी के साथियों ने कबूल किया कि कैसे वे उनके अभियान और डीएनसी के साथ समन्वय स्थापित करने में कामयाब रहे जिसे कम से कम ट्रम्प अभियान को परेशानी में डालने और भ्रष्टों को उकसाने के लिए उकसाने वाले कदम के रूप में वर्णित किया जा सकता है। मुख्यधारा की मीडिया ने ट्रम्प की रैलियों के बाहर हिंसक विस्फोटों के लिए उन्हें दोषी ठहराते हुए घटनाओं को कवर किया। जैसा कि हमने पिछले दो वीडियो से सीखा यह क्लिंटन अभियान ही था जिसने ट्रम्प की रैलियों के बाहर हिंसा पैदा करने के लिए एजेंट उकसाने वालों का इस्तेमाल किया था क्योंकि इन एजेंटों ने खुद को बर्नी सैंडर्स के अनुयायियों के रूप में प्रच्छन्न किया था। अपने दूसरे वीडियो में जेम्स ओकीफ ने दिखाया कि कैसे डीएनसी और हिलेरी क्लिंटन अभियान बड़े पैमाने पर मतदाता धोखाधड़ी को सफलतापूर्वक अंजाम देने के लिए गुर्गों को नियुक्त करते हैं। स्कॉट फोवल जिन्हें तब से निकाल दिया गया है स्वीकार करते हैं कि डेमोक्रेट पचास वर्षों से चुनावों में धांधली कर रहे हैं। अपने पहले वीडियो चुनाव में धांधली - वीडियो I क्लिंटन अभियान और डीएनसी ने ट्रम्प रैलियों में हिंसा भड़काई ओकीफ ने दिखाया कि कैसे डेमोक्रेटिक पार्टी और क्लिंटन अभियान से जुड़े दूसरे और तीसरे पक्ष ट्रम्प समर्थकों पर हिंसा और हमलों के लिए जिम्मेदार थे। पूरे देश में। जैसा कि हमने पहले कहा क्लिंटन अभियान और डेमोक्रेटिक पार्टी पर छाया भ्रष्टाचार का पर्दा गिरना जारी है और लोग आसानी से देख सकते हैं कि उनके नकली प्रगतिशील और नकली उदारवादी मुखौटों के पीछे क्या है। यह पोस्ट हिलेरी क्लिंटन के लिए डक एंड गो वीडियो का समय है पहली बार द स्लीथ जर्नल में दिखाई दी।"

emb = get_hinvec_embedding(test_news)
pred = clf.predict([emb])[0]

print("Prediction:", "🛑 Fake" if pred == 1 else "✅ Real")

Prediction: 🛑 Fake
